In [1]:
from utils.camera import CameraDisplay
import time
import cv2
import onnxruntime as ort
import torchvision.transforms.functional as tf
import torch

from PIL import Image
from utils.dataloader import image_transform
from utils.viz import plot_predictions
now = time.time()

In [2]:
# Define a callback function (your detection pipeline)
# Make sure to first load all your pipeline code and only at the end init the camera
model_file = "pretrained.onnx"
sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
ort_sess = ort.InferenceSession('data/'+model_file, sess_options=sess_options, providers=['CUDAExecutionProvider'])

def predict(image):
    image = Image.fromarray(image)
    image = image_transform(image)[0]
    image = tf.to_tensor(image)
    image = torch.unsqueeze(image, 0)
    onnx_predictions = ort_sess.run(None, {"input.1": image.numpy()})[0]
    return plot_predictions(onnx_predictions, image, return_array=True)


def callback(image):
    global now

    fps = f"{int(1/(time.time() - now))}"
    now = time.time()
    image = predict(image)
    cv2.putText(image, "fps="+fps, (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (100, 255, 0), 2, cv2.LINE_AA)
    return image

In [3]:
# Initialize the camera with the callback
cam = CameraDisplay(callback)

Initializing camera...


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [4]:
# The camera stream can be started with cam.start()
# The callback gets asynchronously called (can be stopped with cam.stop())
cam.start()

In [5]:
# The camera should always be stopped and released for a new camera is instantiated (calling CameraDisplay(callback) again)
cam.stop()
cam.release()

lol
lol
Camera released
